In [52]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [53]:
#lists the content of your google drive
!ls "/content/drive/My Drive"

'Colab Notebooks'				      groupwork
'Data Mining - Summaries 2018 19.gdoc'		      HRI
'DDD group assignment report - Line chart 1.gsheet'  'Sample upload.txt'
'Getting started.pdf'


In [0]:
link = 'https://drive.google.com/open?id=1DPZZQ43w8brRhbEMolgLqOWKbZbE-IQu' # The shareable link

In [55]:
import numpy
import pandas as pd
written_train =  numpy.load('drive/My Drive/Colab Notebooks/written_train(1).npy', allow_pickle=True)
spoken_train = numpy.load('drive/My Drive/Colab Notebooks/spoken_train(1).npy', allow_pickle=True)
match_train =  numpy.load('drive/My Drive/Colab Notebooks/match_train(1).npy', allow_pickle=True)
feature_amount = spoken_train[0].shape[1]
print("shape written:", written_train.shape)
print("shape spoken", spoken_train.shape)
print("shape spoken indiv:", spoken_train[4].shape)
print(feature_amount)
print(match_train)

shape written: (45000, 784)
shape spoken (45000,)
shape spoken indiv: (38, 13)
13
[False False False ... False False False]


In [56]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_train)):
    maxs.append(spoken_train[i].shape[0])
maxlen_spoken_train = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_train_3d= pad_sequences(spoken_train, maxlen= max(maxs))
print(spoken_train_3d.shape)

spoken_train_2d =spoken_train_3d.reshape(45000, maxlen_spoken_train*feature_amount)

spoken_train_2d


(45000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [0]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ss_scaler = StandardScaler()
spoken_train_mmsc = ss_scaler.fit_transform(spoken_train_2d)
written_train_mmsc = ss_scaler.fit_transform(written_train)

In [58]:
X_train = numpy.hstack([spoken_train_mmsc, written_train_mmsc])
print(X_train.shape)
y_train=match_train
print(y_train.shape)

(45000, 1993)
(45000,)


In [59]:
from imblearn.over_sampling import SMOTE

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '2': {} ".format(sum(y_train==1)))

sm = SMOTE(random_state=4)
X_res, y_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of X: {}'.format(X_res.shape))
print('After OverSampling, the shape of y: {} \n'.format(y_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_res==0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_res==1)))


X= numpy.array(X_res)
y= numpy.array(y_res)


Before OverSampling, counts of label '1': 40461
Before OverSampling, counts of label '2': 4539 
After OverSampling, the shape of X: (80922, 1993)
After OverSampling, the shape of y: (80922,) 

After OverSampling, counts of label '1': 40461
After OverSampling, counts of label '2': 40461


In [60]:
X_train=X
y_train=y
print(y_train.shape)

(80922,)


In [61]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# instantiate labelencoder object
le = LabelEncoder()
y_train = le.fit_transform(y)
print(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = y_train.reshape(len(y_train), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)
print(y_train)

[0 0 0 ... 1 1 1]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# LSTM

In [0]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [63]:
 print(X_train.shape)
X_train = X_train.reshape(80922,1993,1)
y_train = y_train.reshape(80922,2)

(80922, 1993)


In [64]:
print(X_train.shape)
print(y_train.shape)

(80922, 1993, 1)
(80922, 2)


In [0]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error

In [66]:
model_m = Sequential()
model_m.add(Conv1D(32, 10, activation='relu', input_shape=(1993, 1)))
model_m.add(MaxPooling1D(10))
model_m.add(Conv1D(80, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(rate=0.5))
model_m.add(Dense(1287, activation='relu'))
model_m.add(Dropout(rate=0.5))
model_m.add(Dense(644, activation='relu'))
model_m.add(Dropout(rate=0.5))
model_m.add(Dense(2, activation='softmax'))
print(model_m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 1984, 32)          352       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 198, 32)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 189, 80)           25680     
_________________________________________________________________
global_average_pooling1d_4 ( (None, 80)                0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1287)              104247    
_________________________________________________________________
dropout_9 (Dropout)          (None, 1287)              0         
__________

In [67]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model_m.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 129
EPOCHS = 50

history = model_m.fit(X_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=1,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)
print(history.history.keys())

Train on 64737 samples, validate on 16185 samples
Epoch 1/1
64737/64737 [==============================] - 140s 2ms/step - loss: 0.6242 - acc: 0.6645 - val_loss: 0.1062 - val_acc: 0.9705
dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


# TESTEN

In [0]:
written_test =numpy.load('drive/My Drive/Colab Notebooks/written_test(1).npy', allow_pickle=True)
spoken_test =numpy.load('drive/My Drive/Colab Notebooks/spoken_test(1).npy', allow_pickle=True)

In [69]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_test)):
    maxs.append(spoken_test[i].shape[0])
maxlen_spoken_test = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_test_3d= pad_sequences(spoken_test, maxlen= max(maxs))
print(spoken_test_3d.shape)

spoken_test_2d =spoken_test_3d.reshape(15000, maxlen_spoken_test*feature_amount)

spoken_test_2d


(15000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  1],
       ...,
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [0]:
mm_scaler = StandardScaler()
spoken_test_mmsc = mm_scaler.fit_transform(spoken_test_2d)
written_test_mmsc = mm_scaler.fit_transform(written_test)


In [0]:
X_test = numpy.hstack([spoken_test_mmsc, written_test_mmsc])
X_test = X_test.reshape(15000,1993,1)

In [72]:
from sklearn.metrics import accuracy_score

y_pred = model_m.predict_classes(X_test, verbose=1)

print(y_pred)

print(1 in y_pred)

15000/15000 [==============================] - 13s 851us/step
[1 1 1 ... 1 1 1]
True


In [0]:
numpy.save("attempt97.npy", y_pred)